# 악성 댓글 구분하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 데이터 읽어오기

In [3]:
data = pd.read_csv("/content/drive/MyDrive/data/common_data.csv", delimiter=',', encoding='utf-8')
data = data.astype({'분쟁유발':int})

In [4]:
data[:5]

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타혐오,악플/욕설,clean,분쟁유발,교정문장,띄어쓰기수정문장
0,문재앙은 김정은의 운전사도 안되는 똥휴지다 개돼지들이 뽑은 재앙민국이 망해간다,0,0,0,0,0,0,0,0,1,0,1,물 재앙은 김정은의 운전사도 안되는 똥 휴지다 개돼지들이 뽑은 재앙 민국이 망해간다,문재앙은 김정은의 운전사도 안 되는 똥 휴지 다 개 돼지들이 뽑은 재앙민국이 망해간다
1,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이제보니 장난아니고 일 재앙...,0,0,0,0,0,0,0,0,0,0,1,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이제 보니 장난 아니고 일 ...,근데 사람들이 우스갯소리로 일 재앙이라고 했는데 이 제보니 장난 아니고 일 재앙은 ...
2,공무뭔들도 정권 바뀐다는걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이지 ...,0,0,0,0,0,0,0,0,1,0,1,공무 뭔들도 정권 바뀐다는 걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이...,공무뭔들도 정권 바뀐다는 걸 아는구나 이놈들은 원래 연줄이 밥줄이라 눈치는 백단이지...
3,조옷도 모르는게 가만히 쳐있어 그냥 주둥이 나불대지말구 이 좌빨 니들 얘기하는...,0,0,0,0,0,0,0,0,0,0,1,저 옷도 모르는 게 가만히 쳐있어 그냥 주둥이 나불대지 말고 이 좌빨 너희들 ...,조옷도 모르는 게 가만히 쳐 있어 그냥 주둥이 나 불대지 말구 이 좌빨 니들 얘기하...
4,토착빨갱이같으니라고,0,0,0,0,0,0,0,0,0,0,1,토착 빨갱이 같으니라고,토착 빨갱이 같으니라고


In [5]:
data.shape

(47409, 14)

### 텍스트 전처리

In [6]:
data.문장 = data.문장.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
data.문장.replace('', np.nan, inplace=True)
data.dropna(how='any', inplace=True)
data.shape
# 0개 제거

(47409, 14)

### 형태소 분석 & 정수 인코딩


In [7]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 15.13 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 63.7 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-14 01:44:24--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (b

In [8]:
from konlpy.tag import Mecab
from tqdm.notebook import tqdm

mecab = Mecab()
f_stopwords = open(f"/content/drive/MyDrive/data/stopwords.txt", encoding='utf-8')

stopwords=[]
for line in f_stopwords.readlines():
  stopwords.append(line.rstrip())

stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [9]:
X = []
#y = []

for i in tqdm(range(len(data.문장))):
  morphs = mecab.morphs(data.문장[i])
  tmp_X = [word for word in morphs if word not in stopwords]
  X.append(tmp_X)
  #y.append(data.iloc[i,1:-2])

# for i in tqdm(range(len(data.교정문장))):
#   morphs = mecab.morphs(data.교정문장[i])
#   tmp_X = [word for word in morphs if word not in stopwords]
#   X.append(tmp_X)
#   #y.append(data.iloc[i,1:-2])

# for i in tqdm(range(len(data.띄어쓰기수정문장))):
#   morphs = mecab.morphs(data.띄어쓰기수정문장[i])
#   tmp_X = [word for word in morphs if word not in stopwords]
#   X.append(tmp_X)
#   #y.append(data.iloc[i,1:-2])

  0%|          | 0/47409 [00:00<?, ?it/s]

In [10]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data.iloc[:,1:-2], test_size=0.2,  random_state=seed)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(X_train)

threshold = 10
total_cnt = len(t.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in t.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 36706
등장 빈도가 9번 이하인 희귀 단어의 수: 30098
단어 집합에서 희귀 단어의 비율: 81.99749359777692
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 8.981322285695123


In [13]:
# 모든 단어를 사용하기로 결정
vocab_size = total_cnt + 2
print(vocab_size)

36708


In [14]:
t = Tokenizer(num_words=vocab_size, oov_token='OOV')
t.fit_on_texts(X)
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

### 패딩

In [15]:
# 데이터의 최대/평균 길이
max(len(s) for s in X_train), sum(map(len, X_train)) / len(X_train)

(158, 20.44213884567722)

In [16]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [17]:
max_len = 70
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 70 이하인 샘플의 비율: 96.63036886650671


In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((37927, 70), (9482, 70))

In [19]:
y_train = y_train.values
y_test = y_test.values
y_train.shape, y_test.shape

((37927, 11), (9482, 11))

### 모델 생성 및 분류하기

#### 1. LSTM

In [20]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [21]:
model1 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    LSTM(128),
    Dense(11, activation='softmax')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 128)           4698624   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 11)                1419      
                                                                 
Total params: 4,831,627
Trainable params: 4,831,627
Non-trainable params: 0
_________________________________________________________________


In [22]:
model1.compile('adam', 'categorical_crossentropy', ['accuracy'])
model1_path = 'unsmile_lstm.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [23]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc1, es1]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.6909 - accuracy: 0.1837
Epoch 1: val_loss improved from inf to 2.33831, saving model to unsmile_lstm.h5
475/475 [==============================] - 16s 12ms/step - loss: 2.6910 - accuracy: 0.1837 - val_loss: 2.3383 - val_accuracy: 0.3104
Epoch 2/100
474/475 [============================>.] - ETA: 0s - loss: 2.3186 - accuracy: 0.3357
Epoch 2: val_loss improved from 2.33831 to 2.26179, saving model to unsmile_lstm.h5
475/475 [==============================] - 5s 11ms/step - loss: 2.3186 - accuracy: 0.3357 - val_loss: 2.2618 - val_accuracy: 0.3654
Epoch 3/100
473/475 [============================>.] - ETA: 0s - loss: 2.1086 - accuracy: 0.4272
Epoch 3: val_loss improved from 2.26179 to 2.20157, saving model to unsmile_lstm.h5
475/475 [==============================] - 6s 13ms/step - loss: 2.1088 - accuracy: 0.4273 - val_loss: 2.2016 - val_accuracy: 0.4558
Epoch 4/100
475/475 [==============================] - ETA: 0s -

In [24]:
best_model1 = load_model(model1_path)
loss1, acc1 = best_model1.evaluate(X_test, y_test)
loss1, acc1

297/297 [==============================] - 2s 4ms/step - loss: 2.2211 - accuracy: 0.4587


(2.2210500240325928, 0.45865851640701294)

#### 2. Conv1D + LSTM

In [25]:
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D

In [26]:
model2 = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    LSTM(128),
    Dense(11, activation='softmax')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 70, 128)           4698624   
                                                                 
 dropout (Dropout)           (None, 70, 128)           0         
                                                                 
 conv1d (Conv1D)             (None, 68, 256)           98560     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 17, 256)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense_1 (Dense)             (None, 11)                1419      
                                                      

In [27]:
model2.compile('adam', 'categorical_crossentropy', ['accuracy'])
model2_path = 'unsmile_conv1d_lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [28]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc2, es2]
)

Epoch 1/100
475/475 [==============================] - ETA: 0s - loss: 2.2093 - accuracy: 0.4094
Epoch 1: val_loss improved from inf to 1.62341, saving model to unsmile_conv1d_lstm.h5
475/475 [==============================] - 14s 13ms/step - loss: 2.2093 - accuracy: 0.4094 - val_loss: 1.6234 - val_accuracy: 0.6674
Epoch 2/100
472/475 [============================>.] - ETA: 0s - loss: 1.5445 - accuracy: 0.7067
Epoch 2: val_loss did not improve from 1.62341
475/475 [==============================] - 4s 9ms/step - loss: 1.5445 - accuracy: 0.7066 - val_loss: 1.6262 - val_accuracy: 0.6902
Epoch 3/100
471/475 [============================>.] - ETA: 0s - loss: 1.5085 - accuracy: 0.7391
Epoch 3: val_loss did not improve from 1.62341
475/475 [==============================] - 4s 9ms/step - loss: 1.5092 - accuracy: 0.7387 - val_loss: 1.9873 - val_accuracy: 0.6901
Epoch 4/100
470/475 [============================>.] - ETA: 0s - loss: 1.5457 - accuracy: 0.7548
Epoch 4: val_loss did not improve fr

In [29]:
best_model2 = load_model(model2_path)
loss2, acc2 = best_model2.evaluate(X_test, y_test)
loss2, acc2

297/297 [==============================] - 2s 4ms/step - loss: 1.6358 - accuracy: 0.6653


(1.6357702016830444, 0.6652604937553406)

#### 3. BiLSTM

In [30]:
from tensorflow.keras.layers import Bidirectional, GlobalMaxPooling1D

In [31]:
model3 = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(11, activation='softmax')
])

model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 70, 128)           4698624   
                                                                 
 bidirectional (Bidirectiona  (None, 70, 256)          263168    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 11)                2827      
                                                                 
Total params: 4,964,619
Trainable params: 4,964,619
Non-trainable params: 0
_________________________________________________________________


In [32]:
model3.compile('adam', 'categorical_crossentropy', ['accuracy'])
model3_path = 'unsmile_bilstm.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [33]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc3, es3]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.4908 - accuracy: 0.3142
Epoch 1: val_loss improved from inf to 1.84957, saving model to unsmile_bilstm.h5
475/475 [==============================] - 12s 18ms/step - loss: 2.4908 - accuracy: 0.3142 - val_loss: 1.8496 - val_accuracy: 0.5985
Epoch 2/100
474/475 [============================>.] - ETA: 0s - loss: 1.6688 - accuracy: 0.6378
Epoch 2: val_loss improved from 1.84957 to 1.66098, saving model to unsmile_bilstm.h5
475/475 [==============================] - 8s 17ms/step - loss: 1.6688 - accuracy: 0.6378 - val_loss: 1.6610 - val_accuracy: 0.6378
Epoch 3/100
472/475 [============================>.] - ETA: 0s - loss: 1.5545 - accuracy: 0.6809
Epoch 3: val_loss did not improve from 1.66098
475/475 [==============================] - 8s 17ms/step - loss: 1.5544 - accuracy: 0.6808 - val_loss: 1.7035 - val_accuracy: 0.6649
Epoch 4/100
472/475 [============================>.] - ETA: 0s - loss: 1.4705 - accuracy: 0.7272


In [34]:
best_model3 = load_model(model3_path)
loss3, acc3 = best_model3.evaluate(X_test, y_test)
loss3, acc3

297/297 [==============================] - 2s 6ms/step - loss: 1.6761 - accuracy: 0.6394


(1.6760666370391846, 0.6394220590591431)

#### 4. Conv1D + BiLSTM

In [35]:
model4 = Sequential([                 
                     
    Embedding(vocab_size, 128, input_length=max_len),
    Dropout(0.2),
    Conv1D(256, 3, padding='valid', activation='relu', strides=1),
    MaxPooling1D(pool_size=4),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(11, activation='softmax')
])

model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 70, 128)           4698624   
                                                                 
 dropout_1 (Dropout)         (None, 70, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 68, 256)           98560     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 17, 256)          0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (None, 17, 256)          394240    
 nal)                                                            
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)             

In [36]:
model4.compile('adam', 'categorical_crossentropy', ['accuracy'])
model4_path = 'unsmile_conv1d_bilstm.h5'
mc4 = ModelCheckpoint(model4_path, verbose=1, save_best_only=True)
es4 = EarlyStopping(patience=5)

In [37]:
hist4 = model4.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc4, es4]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.6951 - accuracy: 0.1767
Epoch 1: val_loss improved from inf to 2.41157, saving model to unsmile_conv1d_bilstm.h5
475/475 [==============================] - 14s 23ms/step - loss: 2.6951 - accuracy: 0.1768 - val_loss: 2.4116 - val_accuracy: 0.2717
Epoch 2/100
474/475 [============================>.] - ETA: 0s - loss: 2.0586 - accuracy: 0.4634
Epoch 2: val_loss improved from 2.41157 to 1.92388, saving model to unsmile_conv1d_bilstm.h5
475/475 [==============================] - 9s 19ms/step - loss: 2.0586 - accuracy: 0.4634 - val_loss: 1.9239 - val_accuracy: 0.5289
Epoch 3/100
474/475 [============================>.] - ETA: 0s - loss: 1.7348 - accuracy: 0.6285
Epoch 3: val_loss did not improve from 1.92388
475/475 [==============================] - 7s 15ms/step - loss: 1.7350 - accuracy: 0.6285 - val_loss: 1.9472 - val_accuracy: 0.6160
Epoch 4/100
474/475 [============================>.] - ETA: 0s - loss: 1.6711 - acc

In [38]:
best_model4 = load_model(model4_path)
loss4, acc4 = best_model4.evaluate(X_test, y_test)
loss4, acc4

297/297 [==============================] - 2s 5ms/step - loss: 1.9231 - accuracy: 0.5253


(1.9230740070343018, 0.5253111124038696)

#### 5. GRU

In [39]:
from tensorflow.keras.layers import GRU

In [40]:
model5 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    GRU(128),
    Dense(11, activation='softmax')
])
model5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 70, 128)           4698624   
                                                                 
 gru (GRU)                   (None, 128)               99072     
                                                                 
 dense_4 (Dense)             (None, 11)                1419      
                                                                 
Total params: 4,799,115
Trainable params: 4,799,115
Non-trainable params: 0
_________________________________________________________________


In [41]:
model5.compile('adam', 'categorical_crossentropy', ['accuracy'])
model5_path = 'unsmile_gru.h5'
mc5 = ModelCheckpoint(model5_path, verbose=1, save_best_only=True)
es5 = EarlyStopping(patience=5)

In [42]:
hist5 = model5.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc5, es5]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.9694 - accuracy: 0.1171
Epoch 1: val_loss improved from inf to 2.95524, saving model to unsmile_gru.h5
475/475 [==============================] - 10s 17ms/step - loss: 2.9695 - accuracy: 0.1171 - val_loss: 2.9552 - val_accuracy: 0.1144
Epoch 2/100
474/475 [============================>.] - ETA: 0s - loss: 2.8554 - accuracy: 0.1551
Epoch 2: val_loss improved from 2.95524 to 2.80383, saving model to unsmile_gru.h5
475/475 [==============================] - 7s 16ms/step - loss: 2.8555 - accuracy: 0.1551 - val_loss: 2.8038 - val_accuracy: 0.1762
Epoch 3/100
474/475 [============================>.] - ETA: 0s - loss: 2.7857 - accuracy: 0.1774
Epoch 3: val_loss improved from 2.80383 to 2.79107, saving model to unsmile_gru.h5
475/475 [==============================] - 7s 15ms/step - loss: 2.7858 - accuracy: 0.1774 - val_loss: 2.7911 - val_accuracy: 0.1760
Epoch 4/100
474/475 [============================>.] - ETA: 0s - lo

In [43]:
best_model5 = load_model(model5_path)
loss5, acc5 = best_model5.evaluate(X_test, y_test)
loss5, acc5

297/297 [==============================] - 2s 4ms/step - loss: 2.6535 - accuracy: 0.2198


(2.6534969806671143, 0.21978485584259033)

#### 6. Conv1D + GRU

In [44]:
model6 = Sequential([ 
    Embedding(vocab_size, 128, input_length=max_len),
    GRU(128),
    Dense(11, activation='softmax')
])
model6.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 70, 128)           4698624   
                                                                 
 gru_1 (GRU)                 (None, 128)               99072     
                                                                 
 dense_5 (Dense)             (None, 11)                1419      
                                                                 
Total params: 4,799,115
Trainable params: 4,799,115
Non-trainable params: 0
_________________________________________________________________


In [45]:
model6.compile('adam', 'categorical_crossentropy', ['accuracy'])
model6_path = 'unsmile_conv1d_gru.h5'
mc6 = ModelCheckpoint(model6_path, verbose=1, save_best_only=True)
es6 = EarlyStopping(patience=5)

In [46]:
hist6 = model6.fit(
    X_train, y_train, validation_split=0.2,
    epochs=100, batch_size=64, callbacks=[mc6, es6]
)

Epoch 1/100
474/475 [============================>.] - ETA: 0s - loss: 2.8661 - accuracy: 0.1576
Epoch 1: val_loss improved from inf to 2.56459, saving model to unsmile_conv1d_gru.h5
475/475 [==============================] - 7s 12ms/step - loss: 2.8661 - accuracy: 0.1576 - val_loss: 2.5646 - val_accuracy: 0.2973
Epoch 2/100
472/475 [============================>.] - ETA: 0s - loss: 2.3697 - accuracy: 0.3293
Epoch 2: val_loss improved from 2.56459 to 2.30205, saving model to unsmile_conv1d_gru.h5
475/475 [==============================] - 5s 11ms/step - loss: 2.3697 - accuracy: 0.3294 - val_loss: 2.3021 - val_accuracy: 0.3518
Epoch 3/100
471/475 [============================>.] - ETA: 0s - loss: 2.2243 - accuracy: 0.3768
Epoch 3: val_loss improved from 2.30205 to 2.23614, saving model to unsmile_conv1d_gru.h5
475/475 [==============================] - 5s 11ms/step - loss: 2.2237 - accuracy: 0.3771 - val_loss: 2.2361 - val_accuracy: 0.3760
Epoch 4/100
472/475 [==========================

In [47]:
best_model6 = load_model(model6_path)
loss6, acc6 = best_model6.evaluate(X_test, y_test)
loss6, acc6

297/297 [==============================] - 2s 4ms/step - loss: 2.2369 - accuracy: 0.3684


(2.2369418144226074, 0.36838218569755554)

### 정확도 비교

In [48]:
print(f"LSTM : {loss1} / {acc1}")
print(f"Conv1D + LSTM : {loss2} / {acc2}")
print(f"BiLSTM : {loss3} / {acc3}")
print(f"Conv1D + BiLSTM : {loss4} / {acc4}")
print(f"GRU : {loss5} / {acc5}")
print(f"Conv1D + GRU : {loss6} / {acc6}")

LSTM : 2.2210500240325928 / 0.45865851640701294
Conv1D + LSTM : 1.6357702016830444 / 0.6652604937553406
BiLSTM : 1.6760666370391846 / 0.6394220590591431
Conv1D + BiLSTM : 1.9230740070343018 / 0.5253111124038696
GRU : 2.6534969806671143 / 0.21978485584259033
Conv1D + GRU : 2.2369418144226074 / 0.36838218569755554


### 실제 데이터 예측

In [49]:
valid_df = pd.read_csv("/content/drive/MyDrive/data/test_multi_spellcheck.csv", delimiter=',', encoding='utf-8')

In [50]:
import re
def unsmile_predict(model, new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = t.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  predicted = model.predict(pad_new) # 예측

  result=[]
  columns = ['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '분쟁유발']
  for i in range(len(predicted[0])):
    if predicted[0][i]>0.1:
      print(f"{columns[i]} ({predicted[0][i]*100:.2f}%)")
  return predicted

In [51]:
columns = ['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '분쟁유발']
for i in range(5,10):
  print(valid_df.문장[i])
  print("\n예측 결과 : ")
  result=unsmile_predict(best_model3, valid_df.문장[i])
  print("\n실제 결과 : ")
  for c in columns:
    if valid_df[c][i]==1:
      print(c)
  print("==========")
  print(result)
  print("==========")

년뒤 윤서인은 분명히 재평가될것임  말하나하나가 틀린게없음

예측 결과 : 
여성/가족 (17.24%)
남성 (12.85%)
성소수자 (14.48%)
악플/욕설 (28.55%)
clean (11.30%)

실제 결과 : 
분쟁유발
[[0.17240506 0.12846217 0.1447991  0.07144325 0.02458918 0.01305443
  0.01248365 0.01454739 0.28546277 0.1129634  0.01978964]]
년만에 재미를 느끼는 프로였는데왜 니들때문에 폐지를해야되냐

예측 결과 : 
인종/국적 (11.90%)
악플/욕설 (33.27%)
clean (32.82%)

실제 결과 : 
clean
[[0.02723202 0.00775398 0.06224868 0.11898019 0.02867773 0.01056509
  0.0525007  0.01846044 0.33266887 0.32818666 0.01272566]]
년차방탄팬인데 우리방탄처럼 성공은못하겠지만 일단 방탄의 부하가되고싶다는거니 이름기억은해둠ㅇㅇ

예측 결과 : 
악플/욕설 (24.96%)
clean (44.71%)

실제 결과 : 
분쟁유발
[[0.03435636 0.02174529 0.06109842 0.07816396 0.01652567 0.00773993
  0.04419023 0.02814673 0.249649   0.44706893 0.01131553]]
살 차이가 넘을텐데 부부라고  무슨 내용인지 긍금하네

예측 결과 : 
악플/욕설 (16.39%)
clean (58.10%)

실제 결과 : 
분쟁유발
[[0.03083974 0.02081712 0.06833695 0.06479942 0.01085215 0.00434666
  0.04053219 0.00919255 0.16391806 0.5809981  0.00536704]]
살 차이면 요즘 세상에 놀랄일도 아니네요

예측 결과 : 
악플/욕설 (16.92%)
clean (57.98%)

실제 

In [52]:
print("LSTM 예측 결과 : ")
unsmile_predict(best_model1, "이래서 여자는 게임을 하면 안된다")
print("==========")
print("Conv1D + LSTM 예측 결과 : ")
unsmile_predict(best_model2, "이래서 여자는 게임을 하면 안된다")
print("==========")
print("BiLSTM 예측 결과 : ")
unsmile_predict(best_model3, "이래서 여자는 게임을 하면 안된다")
print("==========")
print("Conv1D + BiLSTM 예측 결과 : ")
unsmile_predict(best_model4, "이래서 여자는 게임을 하면 안된다")
print("==========")
print("GRU 예측 결과 : ")
unsmile_predict(best_model5, "이래서 여자는 게임을 하면 안된다")
print("==========")
print("Conv1D + GRU 예측 결과 : ")
unsmile_predict(best_model6, "이래서 여자는 게임을 하면 안된다")
print("==========")

LSTM 예측 결과 : 
성소수자 (11.23%)
악플/욕설 (13.29%)
clean (61.60%)
Conv1D + LSTM 예측 결과 : 
여성/가족 (11.94%)
악플/욕설 (37.69%)
clean (24.79%)
BiLSTM 예측 결과 : 
여성/가족 (15.39%)
성소수자 (12.65%)
악플/욕설 (26.05%)
clean (23.95%)
Conv1D + BiLSTM 예측 결과 : 
악플/욕설 (23.77%)
clean (52.03%)
GRU 예측 결과 : 
여성/가족 (12.05%)
악플/욕설 (25.68%)
clean (34.62%)
Conv1D + GRU 예측 결과 : 
인종/국적 (18.47%)
지역 (16.82%)
종교 (12.87%)
악플/욕설 (19.90%)


In [53]:
best_model1.save('lstm.h5')
best_model2.save('conv1d_lstm.h5')
best_model3.save('bilstm.h5')
best_model4.save('conv1d_bilstm.h5')
best_model5.save('gru.h5')
best_model6.save('conv1d_gru.h5')
#best_model4